In [ ]:
%pip -q install google-genai
# Instalar Framework de agentes do Google
%pip install -q google-adk


import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response


# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


##########################################
# --- Agente 1: Buscador de Destinos --- #
##########################################
def agente_buscador(destino, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um agente de viagens da agência Lima's Tur, uma das maiores agências de viagem do Brasil.
        A sua tarefa é usar a ferramenta de busca do google (google_search)
        para buscar informações relevantes apenas sobre a cidade de destino informada abaixo. Você deve ignorar a cidade de origem.
        Informações tais como pontos turísticos, restaurantes, atrações, acomodações
        o que fazer na cidade de destino informada, melhor época do ano para visitar, etc.
        Foque em informações relevantes, com base na quantidade e entusiasmo das avaliações sobre ela.
        Se um ponto turístico, restaurante ou acomodação tiver poucas notícias ou avaliações entusiasmadas, é possível que não seja tão relevante assim
        e pode ser substituído por outros que tenham mais.
        Essas informações relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        É imprescindível a inclusão dos links para os websites dos pontos turísticos, restaurantes, atrações, acomodações, etc. caso estejam disponíveis.
        """,
        description="Agente que busca informações no Google sobre determinado destino",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Destino: {destino}\nData de hoje: {data_de_hoje}"

    destino_buscado = call_agent(buscador, entrada_do_agente_buscador)
    return destino_buscado

################################################
# --- Agente 2: Planejador da viagem --- #
################################################
def agente_planejador(cidade_destino, estado_destino, cidade_origem, estado_origem):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um planejador de viagens da agência Lima's Tur, uma das maiores agências de viagem do Brasil, especialista em
        encontrar passagens aéreas e terrestres mais baratas, acomodações, restaurantes, passeios, etc. para um destino específico.
        Com base na cidade de destino informada, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano de viagem o mais completo possível,
        com valores em reais (R$) de passagens aéres e terrestres, caso tenham. Você também pode usar o Google (google_search) para encontrar mais
        informações sobre cidade de destino e aprofundar. Esse plano de viagem deverá ser completo e detalhado e deverá levar em consideração a cidade de origem
        e o estado de origem, para traçar a rota e informar os valores de transporte. Informe os valores sempre em reais do Brasil (R$).
        Ao final, você irá escolher as melhores opções com base nas suas pesquisas
        e retornar esse plano de viagem completo, com seus pontos mais relevantes.
        Ao informar as opções, prestar muita atenção na separação das palavras, para que fique mais fácil a leitura.
        """,
        description="Agente que planeja viagens",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Cidade destino:{cidade_destino}\nEstado destino: {estado_destino}\nCidade de Origem: {cidade_origem}\nEstado Origem: {estado_origem}"
    # Executa o agente
    plano_de_viagem = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_viagem

######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar e-mails marketing da agência Lima's Tur.
            Você escreve e-mails marketing para uma empresa de viagens especializada em destinos nacionais.
            Utilize o destino fornecido no plano de destino e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de um e-mail cativante sobre o tema indicado.
            O e-mail deve ser engajador, informativo, e deve despertar no leitor uma vontade de conhecer a cidade,
            com linguagem simples e incluir uma chamada para ação no final.
            """,
        description="Agente redator de e-mails marketing engajadores"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho_gerado = call_agent(redator, entrada_do_agente_redator)
    return rascunho_gerado

##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em e-mails para marketing direto.
            Você tem um público diversificado, desde 18 anos até 70 anos. Use um tom de escrita adequado, envolvente, cativante,
            nem formal demais e nem coloquial demais.
            Revise o rascunho do e-mail abaixo sobre o destino indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado


data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Planejamento de Viagens e criação de E-mail Marketing com 4 Agentes 🚀")




########## Lógica do sistma de agentes ##########

while True:
    cidade_destino = input("❓ Por favor, informe a cidade de destino: ").strip()
    if cidade_destino:
        break
    else:
        print("Você esqueceu de digitar a cidade de destino!")

while True:
    estado_destino = input("Informe o estado de destino: ").strip()
    if estado_destino:
        break
    else:
        print("Você esqueceu de digitar o estado de destino!")

while True:
    cidade_origem = input("Informe a cidade de origem: ").strip()
    if cidade_origem:
        break
    else:
        print("Você esqueceu de digitar a cidade de origem!")

while True:
    estado_origem = input("Informe o estado de origem: ").strip()
    if estado_origem:
        break
    else:
        print("Você esqueceu de digitar o estado de origem!")

# --- Obter o destino do Usuário ---


destino = f"{cidade_destino}, {estado_destino} - {cidade_origem}, {estado_origem}"

if cidade_destino and estado_destino and cidade_origem and estado_origem:
    print(f"Maravilha! Vamos então criar o seu roteiro de viagem para {cidade_destino.upper()} - {estado_destino.upper()}")

    destino_buscado = agente_buscador(destino, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(destino_buscado))
    print("--------------------------------------------------------------")

    plano_de_viagem = agente_planejador(cidade_destino, estado_destino, cidade_origem, estado_origem)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_viagem))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(destino, plano_de_viagem)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(destino, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")